In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy

In [2]:
dataset = pd.read_csv('data.csv')
# dataset

In [3]:
X = dataset.drop('xAttack', axis = 1)
Y = dataset['xAttack']

In [4]:
X = (X - np.mean(X))/np.std(X)
X.shape

(125973, 29)

In [5]:
X = X.iloc[:]
# X_copy = copy.deepcopy(X_train)
# Y_copy = copy.deepcopy(Y_train)
# Y_train
Y=pd.concat([Y,pd.get_dummies(Y,prefix='xAttack')],axis=1).drop(['xAttack'],axis=1)
Y = Y.iloc[:]
# X

In [6]:
class Neural_network(object):
    
    def __init__(self):
        self.input_layer_size = 29
        self.output_layer_size = 29
        self.hidden_layer_size1 = 70
        self.bottleneck_layer_size = 14
        self.hidden_layer_size2 = 70
        self.hidden_layers = 2
        self.weights = {}
        self.dweights = {}
        self.Z = {}
        self.epoch = 50
        self.ita = 0.001
        self.a = {}
        self.Y_hat = np.array([])
        self.delta = {}
        
    def initialization(self):
        
        self.weights[0] = np.random.randn(self.input_layer_size,self.hidden_layer_size1)*0.01
        self.weights[1] = np.random.randn(self.hidden_layer_size1,self.bottleneck_layer_size)*0.01
        self.weights[2] = np.random.randn(self.bottleneck_layer_size,self.hidden_layer_size2)*0.01
        self.weights[3] = np.random.randn(self.hidden_layer_size2,self.output_layer_size)*0.01
            
    
    def softmax(self,a):
        tsum = np.sum(a,axis = 1,keepdims = True)
        return np.divide(a,tsum)
    
    
    def error(self,y):
#         return sum(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))/y.shape[0]
        return np.mean(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))
    
    def derivative_sigmoid(self,layer):
        derivative = layer*(1-layer)
        return derivative
    
   
    def deltas(self,y):
        w = self.a[4]
        error = (y - self.Y_hat)
        self.delta[4] = -1 * error * self.derivative_sigmoid(w)
        xx = self.a[3].T
        yy = self.delta[4]
        self.dweights[3] = np.dot(xx , yy)
        
        for i in np.arange(3,0,-1):
            aa = self.delta[i+1]
            bb = self.weights[i].T
            deri = self.derivative_sigmoid(self.a[i])
            self.delta[i] = ( np.dot(aa, bb))*deri
            self.dweights[i-1] = np.dot(self.a[i-1].T , (self.delta[i]))  
    
    def sigmoid(self,x):
        x = -x
        return 1 / (1 + np.exp(x))
    
    
    
    def forward_propagation(self, X):
        self.a[0]=np.array(X)
        self.Z[0] = np.dot(self.a[0],self.weights[0])
        self.a[1] = self.sigmoid(self.Z[0])
        
        self.Z[1] = np.dot(self.a[1],self.weights[1])
        self.a[2] = self.sigmoid(self.Z[1])
        
        self.Z[2] = np.dot(self.a[2],self.weights[2])
        self.a[3] = self.sigmoid(self.Z[2])
        
        self.Z[3] = np.dot(self.a[3],self.weights[3])
        self.a[4] = self.sigmoid(self.Z[3])
        
        self.Y_hat = self.softmax(self.a[4])
        
   
    def gradient_descent(self,X_train,Y_train):

        k = 0
        while k <= 120000:
            for i in range(self.epoch):
                Y = Y_train[k:k+5000]
                X = X_train[k:k+5000]
                self.forward_propagation(X)
                self.deltas(Y)
                for j in range(0,4):
                    lrweights = self.ita*self.dweights[j]
                    self.weights[j] = self.weights[j] - lrweights
            print("Final Error : ",self.error(Y))
            k = k + 5000

        
    def predict(self,X):
        self.a[0] = np.array(X)
        self.Z[0] = np.dot(self.a[0],self.weights[0])
        self.a[1] = self.sigmoid(self.Z[0])
        self.Z[1] = np.dot(self.a[1],self.weights[1])
        self.a[2] = self.sigmoid(self.Z[1])
        return self.a[2]
    
            

In [7]:
k = Neural_network()
k.initialization()
k.gradient_descent(X,X)

Final Error :  12.373599243932535
Final Error :  12.090544478232074
Final Error :  12.655521692699855
Final Error :  12.829831489774682
Final Error :  14.333586369100498
Final Error :  13.015716213691432
Final Error :  13.0809467016704
Final Error :  13.166969052324845
Final Error :  17.815139442376587
Final Error :  12.86324817176206
Final Error :  11.42246244457383
Final Error :  30.723822867753253
Final Error :  25.31203492212576
Final Error :  13.943101363263553
Final Error :  12.193534443124602
Final Error :  12.636816325695007
Final Error :  13.773782265968906
Final Error :  12.779623924804811
Final Error :  13.821714573389746
Final Error :  12.490435020642431
Final Error :  13.347333319789456
Final Error :  12.144586368901507
Final Error :  13.096734036452588
Final Error :  12.476961524432577
Final Error :  13.684460778420197


In [8]:
# X_test = X_test[0:1000]
# Y_test = Y_test[0:1000]

finaldata=k.predict(X)
finaldata=pd.DataFrame(finaldata)
df_final = pd.concat([finaldata,dataset['xAttack']],axis=1)

In [9]:
# df_final

In [10]:
df_final.to_csv("data_final_deep.csv",index=False)